In [1]:
import os
import sys
from datetime import datetime


from config import MAILSLURP_API
import mailslurp_client
from email.utils import formatdate
from email.message import EmailMessage
import email.header
from mailslurp_client import ApiException, Email, EmailPreview



print(mailslurp_client.__version__)


15.5.6


# Utils

In [55]:
def wrap_email(email_mailslurp: mailslurp_client.Email) -> EmailMessage:
    #raise NotImplementedError(f"wrap_email method not yet implemented for MailSlurp(because ran out of API credits!!!)")
    msg = EmailMessage()
    msg['Subject'] = email_mailslurp.subject
    msg['From'] = email_mailslurp._from
    msg['To'] = email_mailslurp._to
    msg['Date'] = formatdate(email_mailslurp.created_at.timestamp())
    msg.set_content(email_mailslurp.body)
    return msg  # returns the Email Object


# Setup


## Configuration

In [4]:
# set up required MailSlurp Configuration
configuration = mailslurp_client.Configuration()
configuration.api_key['x-api-key'] = MAILSLURP_API

# create an API client
api_client = mailslurp_client.ApiClient(configuration)

# create an InboxControllerApi  # See: https://docs.mailslurp.com/ruby/docs/InboxControllerApi/
inbox_controller = mailslurp_client.InboxControllerApi(api_client)
waitfor_controller = mailslurp_client.WaitForControllerApi(api_client)
email_controller = mailslurp_client.EmailControllerApi(api_client)


# Inbox Fuctions

## Show all inboxes

In [ ]:
inbox_controller.get_all_inboxes()
# inbox_controller.get_all_inboxes_with_http_info()

## Number of Inboxes

In [ ]:
inbox_controller.get_inbox_count()

## Flush expired Inboxes

In [ ]:
inbox_controller.flush_expired()

## Set Inbox to first inbox

In [ ]:
this_inbox = inbox_controller.get_all_inboxes().content[0]
print(f"Inbox set to \nemail address: {inbox_controller.get_all_inboxes().content[0].email_address}")

# print(f"Inbox set to \nID: {this_inbox.id}\nemail address:{this_inbox.email_address}"): {this_inbox.id}\nemail address:{this_inbox.email_address}")

## Set Inbox to specific inbox

In [5]:
this_inbox_id = "aadaf10e-800e-4408-b982-685b13d6ca7a"

try:
    this_inbox = inbox_controller.get_inbox(this_inbox_id)
    print(f"Inbox set to \nID: {this_inbox.id}\nemail address: {this_inbox.email_address}")
except ApiException:
    print(f"ApiError: likely that the inbox_id: {this_inbox_id} is incorrect")


Inbox set to 
ID: aadaf10e-800e-4408-b982-685b13d6ca7a
email address: aadaf10e-800e-4408-b982-685b13d6ca7a@mailslurp.com


## Create  new inbox (with random domain)

In [ ]:
inbox_controller.create_inbox(use_domain_pool=True)

## Get emails (emailPreview) from inbox

In [30]:
all_emails = inbox_controller.get_emails(this_inbox.id)
# print((all_emails[0]).__dict__)  # NB: this retutned type mailslup_client.emailPreview
for email in all_emails:
    print(f"{email.id} // {email.subject} // {email._from}")

730f3e94-cd1f-4b49-b627-9259353109ec // test email subjecy // ajanderson1@gmail.com


In [54]:
email = inbox_controller.get_latest_email_in_inbox(this_inbox.id,timeout_millis=56)
wrapped_email = wrap_email(email)
print(wrapped_email.as_string())

Date: Sun, 06 Feb 2022 13:47:32 -0000
Subject: test email subjecy
From: ajanderson1@gmail.com
To: aadaf10e-800e-4408-b982-685b13d6ca7a@mailslurp.com
created_at: 02/06/2022, 13:47:32
Content-Type: text/plain; charset="utf-8"
Content-Transfer-Encoding: 7bit
MIME-Version: 1.0

<div dir="ltr">testting email</div>



## Get email from emailID (from emailPreview)

In [31]:
this_email = email_controller.get_email(all_emails[0].id)
this_email.body

'<div dir="ltr">testting email</div>\r\n'

# Emails

In [ ]:
# list all inboxes
with mailslurp_client.ApiClient(configuration) as api_client:
    inbox_controller = mailslurp_client.InboxControllerApi(api_client)
    inboxes = inbox_controller.get_all_inboxes(page=0)

    # pagination properties
    assert inboxes.total_pages > 0
    assert inboxes.total_elements > 0

    # view contents
    assert len(inboxes.content[0].id) > 0
    for this_inbox in inboxes.content:
        print (this_inbox.id)


In [ ]:
inbox = inboxes.content[0]
# print(inbox.__dict__)
print(help(inbox))